In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Config
from transformers import get_linear_schedule_with_warmup

import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split, RandomSampler, SequentialSampler

import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"
# model_name: ['gpt2', 'gpt2-medium', 'gpt2-large', 'gpt2-xl']
model_name = "gpt2" 
model_save_path = './model'

### Test

In [9]:
configuration = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)

tokenizer = GPT2TokenizerFast.from_pretrained(model_name)

input_sequence = "beef, salt, pepper"
input_ids = tokenizer.encode(input_sequence, return_tensors='pt')

model = model.to(device)
#combine both sampling techniques
sample_outputs = model.generate(input_ids.to(device),
                              do_sample = True, max_length = 120,
                              top_k = 50, top_p = 0.85,
                              num_return_sequences = 3)

for sample in sample_outputs:
    print(tokenizer.decode(sample))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


beef, salt, pepper, garlic, parsley and a small amount of chili powder to taste. This is a recipe that we've been using at the grocery store for quite a while.

If you'd like to make this recipe, please share on social media with the hashtag #yummytomato. I'd love to see what you have to share with us, please take a picture or post a link below.

You'll also need a good quality blender, such as this one I use in the grocery store.

4.0 from 5 votes Print Y
beef, salt, pepper and olive oil.

For this I used black olives and dried fruits. The olives were sweetened with anise.

For this I used black beans and onions, fresh tomatoes, dried figs, basil, garlic and oregano. I then chopped the tomatoes into chunks and baked them in the oven for 45 minutes. Then, I baked the cooked tomatoes in the oven for another 20 minutes. Then I baked the roasted potatoes in the oven for another 15 minutes.

For this I used white beans, dried fruit, parsley and
beef, salt, pepper, garlic, pepper and vinega

### Preprocess

In [ ]:
df_recipes = pd.read_csv('recipes_1000.csv')
df_recipes.reset_index(drop=True, inplace=True)

def form_string(ingredient,instruction):
    s = f"<|startoftext|>Ingredients: {ingredient.strip()}. " \
        f"Instructions: {instruction.strip()}<|endoftext|>"
    return s

data = df_recipes.apply(lambda x:form_string(
    x['ingredients'], x['instructions']), axis=1).to_list()
data[0]


tokenizer = GPT2TokenizerFast.from_pretrained(model_name,
                                              bos_token='<|startoftext|>',
                                              eos_token='<|endoftext|>',
                                              unk_token='<|unknown|>',
                                              pad_token='<|pad|>'
                                             )


### Dataset

In [11]:
batch_size = 2
max_length = 180  

# standard PyTorch approach of loading data in using a Dataset class.
class RecipeDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.input_ids = []
        self.attn_masks = []

        for recipe in data:
            encodings = tokenizer.encode_plus(recipe,
                                              truncation=True,
                                              padding='max_length',
                                              max_length=max_length,
                                              # return a PyTorch tensor
                                              return_tensors='pt'       
                                             )
            self.input_ids.append(torch.squeeze(encodings['input_ids'],0))
            self.attn_masks.append(torch.squeeze(encodings['attention_mask'],0))


    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        return self.input_ids[idx], self.attn_masks[idx]

dataset = RecipeDataset(data, tokenizer)
print(f"input_ids: {dataset[0][0]} attn_masks: {dataset[0][1]}")

 dataset[0]: 
  input_ids: tensor([50257, 41222, 25,  4171, 20853, 11, 19468, 4817,
        16858, 32132, 11, 18873, 13135, 13, 27759, 25, 309, 793,
        4691, 13, 50256, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
        50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259])
   attn_masks: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


### Train

In [ ]:
# Split into training and validation sets
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create the DataLoaders for our training and validation datasets.
# Get training samples in random order.
train_dataloader = DataLoader(
            train_dataset, 
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size # Trains with this batch size.
        )

# Get valiation samples sequentially.
validation_dataloader = DataLoader(
            val_dataset, 
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size # Evaluate with this batch size.
        )

configuration = GPT2Config.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name, config=configuration)
model = model.to(device)
model.resize_token_embeddings(len(tokenizer))

epochs = 3
learning_rate = 2e-5
warmup_steps = 1e2
# to prevent any division by zero in the implementation
epsilon = 1e-8
optim = AdamW(model.parameters(), lr = learning_rate, eps = epsilon)

total_steps = len(train_dataloader) * epochs  # [no batches] x [no epochs]

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optim,
                                            num_warmup_steps=warmup_steps,
                                            num_training_steps=total_steps)

In [ ]:
for epoch_i in range(0, epochs):
    total_train_loss = 0
    model.train() 

    for step, batch in enumerate(train_dataloader): 
        b_input_ids = batch[0].to(device) 
        b_labels    = batch[0].to(device)
        b_masks     = batch[1].to(device) 

        model.zero_grad()
        outputs = model( input_ids = b_input_ids, labels = b_labels,
                         attention_mask = b_masks, token_type_ids = None )

        loss = outputs[0]

        # Get sample every x batches.
        if step % 100 == 0 and not step == 0:
            model.eval()
            print(infer("eggs, flour, butter, sugar"))
            model.train()

        loss.backward()
        optim.step()
        scheduler.step()

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

### Eval

In [13]:
def infer(prompt):
    input = f"<|startoftext|>Ingredients: {prompt.strip()}"
    input = tokenizer(input, return_tensors="pt")
    input_ids      = input["input_ids"]
    attention_mask = input["attention_mask"]

    output = model.generate(input_ids.to(device),
                            attention_mask=attention_mask.to(device),
                            max_new_tokens=max_length,
                            do_sample = True, top_k = 50, top_p = 0.85)
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return output

model = GPT2LMHeadModel.from_pretrained(model_save_path)
tokenizer = GPT2TokenizerFast.from_pretrained(model_save_path)
model.to(device)

infer("eggs, mushroom, butter, sugar")

 Ingredients: eggs, mushroom, butter, sugar, milk, cream, cornstarch, salt, brown sugar, flour, baking powder, baking soda, baking soda.
Instructions: In a large saucepan, bring eggs to boil. Reduce heat and simmer until thickened. Remove from heat and stir in mushroom, butter, sugar, milk, cornstarch, salt, and brown sugar. Add the flour, baking powder and baking soda; stir until well blended. Add the baking soda and stir until completely dissolved. Add milk and cornstarch mixture to the mushroom mixture. Stir well to combine. Divide mixture into 8 equal parts and bake at 350 degrees for 15 minutes. Let stand 5 minutes before serving. Makes 4–6 servings.
